# CHSolver User Tutorial

## Requirements
We assume the following software will be already available prior to installation:
- GCC/10.2.0
- CMake/3.20.1
- OpenMPI/4.0.5
- HDF5/1.10.7
- Python/3.8.6
- FFTW/3.3.8

## Installation
The repository can be git cloned via the command  $\texttt{ git clone --recursive git@github.com:HetSys/CHSolver.git}$

The code requires some external dependancies, which are linked to the repository through the use of git submodules. The bash script $\texttt{build\_deps}$ is provided in order to build and install these locally, within the repository.

Make is used as the main driver for the executable compilation and support. Supported commands are as follows:
- make: Compile and link the chsolver executable
- make clean: Clean the executables, .o and .mod files
- make logpurge: Clean all .log files from the ./logs dir
- make docs: Generate developer documentation (requires a doxygen executable on path, tested with Doxygen/1.8.17)
- make tests: Compile and link a unit tests executable (requires pFUnit to have been installed within the repository)

## Handling JSON input data
JSON files are used by both the Python API and the $\texttt{chsolver}$ executable in order to more easily handle input parameters to the executable. Though it is possible to use the the executable without requiring a JSON file, it is strongly recommended.

The JSON datastructure operates via nesting of key : value pairs. At the root level, the JSON file is subdivided into "runs", where each run contains the full set of parameters required for proper execution of the program. A simple example file is shown below.

```
{
  "default": {
    "L": 1.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -1.0,
    "p1": 1.0,
    "grid_level": 7,
    "grid_type": "r",
    "T": [
      0.0,
      0.05,
      0.1
    ]
  }
}
```

The run, called "default" contains a nested set of parameters ("L", "A", "M", ...) which would all be searched by name during execution of the program. To add alternative runs directly to the file, it is recommended to first copy and paste the default run, rename it, and thn modify the values associated with the parameters.

### Handling the JSON file from Python
The CHData class (from src/dataclass.py) provides an interface to generate and modify JSON files interactively.
A worked example of generating and adding a new run to a json file from the Python API is shown below. 

In [2]:
# PYTHON IMPORTS
import numpy as np
import os
from src.dataclass import CHData

In [38]:

# Generating an instance of CHData will create or open the file given by the fname init arg.
# If the file did not already exist, the default run will be saved to file
dat = CHData(fname="tutorial-data.json")

# Print the input data currently loaded
# By default, this will be the data contained in the "default" run in the
dat.print_rundata()

L = 1.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -1.0
p1 = 1.0
Grid type = r
Grid level = 7
Output Times = [0.   0.05 0.1 ]


In [39]:
# Modify the input data stored in the CHData object
dat.L = 2.0

dat.p0 = -3.0

dat.grid_level = 4

dat.T = np.linspace(0, 0.2, 5)

dat.grid_type = "c"

# Print out the modifications made to the input data
dat.print_rundata()

L = 2.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -3.0
p1 = 1.0
Grid type = c
Grid level = 4
Output Times = [0.   0.05 0.1  0.15 0.2 ]


In [40]:
# Save the input data back down to the file
dat.save_rundata(run_name="modified")

Opening the "tutorial-data.json" file, it should now look like this:
```
{
  "default": {
    "L": 1.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -1.0,
    "p1": 1.0,
    "grid_level": 7,
    "grid_type": "r",
    "T": [
      0.0,
      0.05,
      0.1
    ]
  },
  "modified": {
    "L": 2.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -3.0,
    "p1": 1.0,
    "grid_level": 4,
    "grid_type": "c",
    "T": [
      0.0,
      0.05,
      0.1,
      0.15000000000000002,
      0.2
    ]
  }
}
```

Now we have a file containing multiple runs, we can look into quickly swapping between them.

In [41]:
# Create a new object, opening the same file
dat2 = CHData(fname="tutorial-data.json")

# Print the run names found in the file
print(dat2.run_names)

['default', 'modified']


In [42]:
# Print the currently loaded run data
dat2.print_rundata()

L = 1.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -1.0
p1 = 1.0
Grid type = r
Grid level = 7
Output Times = [0.   0.05 0.1 ]


In [43]:
# Read the data contained in the "modified" datastructure
dat2.read_rundata("modified")

# Verify the new data has been loaded
dat2.print_rundata()

L = 2.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -3.0
p1 = 1.0
Grid type = c
Grid level = 4
Output Times = [0.0, 0.05, 0.1, 0.15000000000000002, 0.2]


## Using the $\texttt{chsolver}$ executable

The $\texttt{chsolver}$ can be built via the Makefile, using the command $\texttt{make}$. The executable comes with a fully featured Command Line Interface in order to allow the program to be flexible to a variety of tasks.

To demonstrate the program for this tutorial, we run shell commands via the python os.system() function.

In [44]:
# OPTIONAL: Compile the chsolver executable (if not already build)
os.system("make clean && make")

rm -f -r -d chsolver  ./obj/*.o ./src/*.mod ./out/** ./doxygen/output/* test_all
make -C tests/unit_tests clean
make[1]: Entering directory '/home/u2180064/Documents/GitHub/CHSolver/tests/unit_tests'
rm -f *.o *.mod *.a  *.inc *.F90
make[1]: Leaving directory '/home/u2180064/Documents/GitHub/CHSolver/tests/unit_tests'
mkdir -p ./obj ./out ./logs
Building obj/face.o
h5fc  -std=f2008 -c -o obj/face.o src/submodules/FACE/src/lib/face.F90 
Building obj/logger_mod.o
h5fc  -std=f2008 -c -o obj/logger_mod.o src/submodules/flogging/src/logging.f90 
Building obj/logging.o
h5fc  -J./src -I./src/submodules/bin/jsonfortran-gnu-8.2.5/lib -Wall -Wextra -Wconversion-extra -std=f2008 -g -fall-intrinsics -c -o obj/logging.o src/logging.f90 
Building obj/globals.o
h5fc  -J./src -I./src/submodules/bin/jsonfortran-gnu-8.2.5/lib -Wall -Wextra -Wconversion-extra -std=f2008 -g -fall-intrinsics -c -o obj/globals.o src/globals.f90 
Building obj/command-line.o
h5fc  -J./src -I./src/submodules/bin/jsonfortran-gn

0

### Basic Usage

In [45]:
# Check the version of the program
version_command = "./chsolver --version"
os.system(version_command)

# Print help text

help_command = "./chsolver -h"
#os.system(help_command)


 CHSolver 1.0


In [4]:
# IO Commands

json_file = "tutorial-data.json"

output_dir = "tutorial-out"
os.makedirs(output_dir, exist_ok=True)

run_name = "modified"

exe = "./chsolver"

# -j <file> will change the json file searched for inputs
json_file_command = f" -j {json_file}"

# -o <dir> will change the location output files are saved to
output_dir_command = f" -o {output_dir}"

# -r <name> will change the name of the run parsed in the JSON for inputs 
run_name_command = f" -r {run_name}"


# Generate the full command
basic_command = exe + json_file_command + output_dir_command + run_name_command

print(basic_command)
os.system(basic_command)

./chsolver -j tutorial-data.json -o tutorial-out -r modified
[May 24 2022 16:44:27][open_json] <info> Reading tutorial-data.json
[May 24 2022 16:44:27][setup_grid] <info> Generating Circle grid
[May 24 2022 16:44:27][solver_1] <info> Solving with fd2
[May 24 2022 16:44:27][solver_ufds2t2] <info> Output at t=  0.000                             
[May 24 2022 16:44:30][solver_ufds2t2] <info> Output at t=  0.100                             
[May 24 2022 16:44:34][solver_ufds2t2] <info> Output at t=  0.200                             
[May 24 2022 16:44:37][solver_ufds2t2] <info> Output at t=  0.300                             
[May 24 2022 16:44:42][solver_ufds2t2] <info> Output at t=  0.400                             


0

In [ ]:
# Print the generated files
print(sorted(os.listdir(output_dir)))

The executable first makes a "metadata.dat" file in the output directory, which contains the input parameters used for the run. It also contains a tables linking the checkpoint number with the simulated time the checkpointed data corresponds to.

The checkpoint files are binary files using the hdf5 storage scheme. They each contain the concentration grid C, as well as the grid state at the previous timestep, and the time interval used for that timestep (NOTE: timestep is not equivalent to the difference between output times, the solvers internally use an adaptive timestep scheme in order to provide stable and accurate outputs).

### Overriding the JSON inputs

In [ ]:
# New values for parameters can be specified from the command line - if all of L, A, M, K, p0, p1, grid_init, grid_level and T 
# are defined from the command line, the json parsing is not done.

A_command = " -A 2.0"

p1_command = " -p1 0.01"

# Arrays in the command line have to be colon separated
T_command = " -t {0.0:0.1:0.2:0.3}"

full_command = basic_command + A_command + p1_command + T_command

print(full_command)
os.system(full_command)

./chsolver -j tutorial-data.json -o tutorial-out -r modified -A 2.0 -p1 0.01 -t {0.0:0.1:0.2:0.3}
[May 24 2022 14:47:42][open_json] <info> Reading tutorial-data.json
[May 24 2022 14:47:42][setup_grid] <info> Generating Circle grid
[May 24 2022 14:47:42][solver_1] <info> Solving with fd2
[May 24 2022 14:47:42][solver_ufds2t2] <info> Output at t=  0.000                             
[May 24 2022 14:47:44][solver_ufds2t2] <info> Output at t=  0.085                             
[May 24 2022 14:47:45][solver_ufds2t2] <info> Output at t=  0.170                             
[May 24 2022 14:47:47][solver_ufds2t2] <info> Output at t=  0.256                             


0

### Advanced commands: Verbosity, easier time array specification

In [ ]:
# Verbosity can be controlled via the -v (verbose) and -q (quiet) commands

#verbosity_command = " -qq"  # Very quiet (ignores warning messages)
verbosity_command = " -q" # Quiet (ignores info messages)
#verbosity_command = " " # No verbosity change
#verbosity_command = " -v" # Verbose (prints trivia messages)
#verbosity_command = " -vv" # Very verbose (prints debug messages)

#timespace_command = " --lin_tspace={0.0:0.2:5}" # Specify a linspace T from 0.0 to 0.2 with 5 outputs
timespace_command = " --log_tspace={0.0:0.2:5}" # Specify a logspace T from 0.0 to 0.2 with 5 outputs

full_command = basic_command + verbosity_command + timespace_command

print(full_command)
os.system(full_command)

./chsolver -j tutorial-data.json -o tutorial-out -r modified -q --log_tspace={0.0:0.2:5}


0

### Restarting from a checkpoint
The metadata and checkpoint files contain enough information about the state at any output to allow the calculation to be resumed from any of these points, in the event of a fatal crash or where the user desired to simulate the system a little further.

In the case of a checkpoint, the input time array

In [5]:
initial_command = basic_command + " --lin_tspace={0.0:0.1:5}"

os.system(initial_command)


#cont_from_time = "--restart_time=0.1" # Can specify a specific time to restart from. Will restart from a checkpoint with a time <= this time

cont_from_checkpoint = " --restart_num=-1" # Restart from a specific checkpoint number, with -1 meaning restart from the last checkpoint
continuation_times = " --lin_tspace{0.0:0.2:10}" # The times from 0.0 to 0.1 should be masked out of this array, as these have already been evaluated

restart_command = basic_command + cont_from_checkpoint + continuation_times

print(restart_command)
os.system(restart_command)

[May 24 2022 16:44:42][open_json] <info> Reading tutorial-data.json
[May 24 2022 16:44:42][setup_grid] <info> Generating Circle grid
[May 24 2022 16:44:42][solver_1] <info> Solving with fd2
[May 24 2022 16:44:42][solver_ufds2t2] <info> Output at t=  0.000                             
[May 24 2022 16:44:43][solver_ufds2t2] <info> Output at t=  0.050                             
[May 24 2022 16:44:45][solver_ufds2t2] <info> Output at t=  0.100                             
[May 24 2022 16:44:47][solver_ufds2t2] <info> Output at t=  0.150                             
[May 24 2022 16:44:49][solver_ufds2t2] <info> Output at t=  0.200                             
./chsolver -j tutorial-data.json -o tutorial-out -r modified --restart_num=-1 --lin_tspace{0.0:0.2:10}
[May 24 2022 16:44:49][open_json] <info> Reading tutorial-data.json
[May 24 2022 16:44:49][solver_2] <info> Solving with fd2
[May 24 2022 16:44:56][solver_ufds2t2] <info> Output at t=  0.300                             
[May 24 2022

0

### Python executable wrapper
For convenience, a simple interface to the executable is provided as a method of the CHData class. It internally calls the executable using the L, A, M, ... attributes and automatically reads the produced output files.

In [6]:
dat = CHData()

dat.grid_level = 3

# Can specify any additional command line args using a cmd_args string
dat.solve(out_dir="out", cmd_args="-v")

print(dat.C[0, :, :]) # Print first timestep

[0.         0.02222222 0.04444444 0.06666667 0.08888889 0.11111111
 0.13333333 0.15555556 0.17777778 0.2       ]
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting L  to    1.0000000000000000
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting A  to    1.0000000000000000
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting M  to   0.25000000000000000
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting K  to    4.0000000000000002E-004
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting p0 to   -1.0000000000000000
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting p1 to    1.0000000000000000
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting level to            3
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting grid initialisation character to r
[May 24 2022 16:51:36][get_input_commands] <trivia> Setting output timesteps to {   0.0000000000000000        2.2222222222222223E-002   4.4444444444444446E-002   6.6666666666666

# Output Files

## Working with output files from Python

The CHData class also provides functionality for working with the output files produced by the $\texttt{chsolver}$ program.

In [ ]:
dat = CHData(fname="tutorial-data.json")

The user can generate an animation of the concentration with time (gif) and a plot of the free energy versus time (png) by calling python3 src/visualisation.py. These will be saved to the main directory.
Should the user wish to change the animation's fps (default 10) or specify the time interval for either of the (default start to finish), for either of the visuals, then the python file can be opened and the arguments to the plot_conc_evol and plot_free_energy functions at the bottom of the file can be changed.